In [1]:
import numpy as np
import pandas as pd

from astropy.io import fits

#fits_image_filename = fits.util.get_testdata_filepath()

In [2]:
_epsilon = 3e-4 

In [3]:
hdul_gev = fits.open('data/gll_psc_v16.fit')
cat_gev = hdul_gev[1].data

In [4]:
names_gev = [
    'ASSOC_TEV', 
    'CLASS1', 
    'GLON', 
    'GLAT', 
    'Variability_Index', 
    'Flux1000', 
    'Flux10000_100000', 
    'Flux1000_3000', 
    'Flux100_300', 
    'Flux3000_10000', 
    'Flux300_1000',  
    'Flux30_100',
    ] 

In [5]:
hdul_tev = fits.open('data/gammacat.fits.gz')
cat_tev = hdul_tev[1].data

In [ ]:
names_tev = [
    'classes', 
    'glat', 
    'glon', 
    'morph_pa', 
    'sed_dnde', 
    'sed_dnde_err', 
    'sed_e_ref', 
    'spec_dnde_1TeV', 
    'spec_dnde_1TeV_err', 
    'spec_eflux_1TeV_10TeV', 
    'spec_eflux_1TeV_10TeV_err', 
    'spec_flux_1TeV', 
    'spec_flux_1TeV_crab', 
    'spec_flux_1TeV_crab_err'
    ]

In [ ]:
def corresponding_bins(cat_gev, cat_tev, epsilon):
    d = {}
    
    class_gev = cat_gev['CLASS1']
    class_tev = cat_tev['classes']
    glat_gev = cat_gev['GLAT']
    glat_tev = cat_tev['glat']
    glon_gev = cat_gev['GLON']
    glon_tev = cat_tev['glon']
    
    for j in range(len(glat_tev)):
        for i in range(len(glat_gev)):
            if ((class_tev[j].find('bin') != -1) 
                & ((class_gev[i].find('BIN') != -1) | (class_gev[i].find('HMB') != -1))
                & (np.abs(glat_gev[i] - glat_tev[j]) < epsilon) 
                & (np.abs(glon_gev[i] - glon_tev[j]) < epsilon)):
                d.update({j : i})
    return d
                
print(corresponding_bins(cat_gev, cat_tev, 1))

In [ ]:
gevToTev = {'BLL': 'blazar', 
            'FRSQ': 'frsq', 
            'HMB': 'bin' , 
            'BIN': 'bin', 
            'GAL': 'galaxy', 
            'PSR': 'psr', 
            'PWN': 'pwn', 
            'SNR': 'snr', 
            '': 'unid'}
tevToGev = {v:k for k, v in gevToTev.items()}

In [ ]:
def common(cat_gev, cat_tev, epsilon):
    """This function looks for the same objects in GeV and TeV catalogs
    
    Return: dictionary with corresponding values
    
    feature1(list) - features of GeV 
    feature2(list) - features of TeV 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}

    class_gev = cat_gev['CLASS1']
    class_tev = cat_tev['classes']
    glat_gev = cat_gev['GLAT']
    glat_tev = cat_tev['glat']
    glon_gev = cat_gev['GLON']
    glon_tev = cat_tev['glon']
    
    for i in range(len(glat_gev)):
        for j in range(len(glat_tev)):
            classGeV = class_gev[i]
            start = -1
            try: 
                start = (class_tev[j].find(gevToTev[classGeV]))
            except KeyError:
                continue
            if (start != -1):
                if ((glat_gev[i] - glat_tev[j] < epsilon) and (glon_gev[i] - glon_tev[j] < epsilon)) :
                    d.update({j : i})
    return d

D = common(cat_gev, cat_tev, 3e-10)
print(D)

In [ ]:
names_common = [
    'glat',
    'glon',
    'morph_pa',
    'sed_dnde',
    'sed_dnde_err',
    'sed_e_ref', 
    'spec_dnde_1TeV', 
    'spec_dnde_1TeV_err', 
    'spec_eflux_1TeV_10TeV', 
    'spec_eflux_1TeV_10TeV_err', 
    'spec_flux_1TeV', 
    'spec_flux_1TeV_crab', 
    'spec_flux_1TeV_crab_err', 
    'ASSOC_TEV', 
    'Variability_Index', 
    'Flux1000', 
    'Flux10000_100000', 
    'Flux1000_3000', 
    'Flux100_300', 
    'Flux3000_10000', 
    'Flux300_1000', 
    'Flux30_100', 
    'Flux1000', 
    'Flux10000_100000', 
    'Flux1000_3000', 
    'Flux100_300', 
    'Flux3000_10000', 
    'Flux300_1000', 
    'Flux30_100', 
    'CLASS1']

In [ ]:
d_coord = {'glon' : 'GLON', 'glat' : 'GLAT'}

In [ ]:
def create_common_data(cat_gev, cat_tev, D, namefinal):
    """The fonction adds objects found both in GeV and TeV.
    
    D(dictionary) - a dictionary with repeated objects from TeV and GeV
    namefinal(list) - names of required features to fill in data
    """
    k = 0
    data = []
    
    for j in D.keys():
        data.append([])
        for i in range(13):
            data[k].append(cat_gev[namefinal[i]][j])
        for i in range(13, len(namefinal)):
            data[k].append(cat_tev[namefinal[i]][D[j]])
        k = k+1
        
    df_common = pd.DataFrame(data = data, columns = namefinal)
        
    return df_common
    #print(str(cat['CLASS1'][D[i]]) + " - " + str(cat2['classes'][i]) + "\n")

In [ ]:
def create_only_tev_data(cat_tev, D, name_tev):
    """The fonction adds objects found only in TeV.
    
    D(dictionary) - a dictionary with repeated objects from TeV and GeV
    name_tev(list) - names of TeV columns
    """
    k = 0
    data = []
    
    for j in range(len(cat_tev)):
        if not(j in D.keys()):
            data.append([])
            for i in range(len(name_tev)):
                data[k].append(cat_tev[name_tev[i]][j])
            k = k+1
            
    df_only_tev = pd.DataFrame(data = data, columns = name_tev)
        
    return df_only_tev

In [ ]:
def create_only_gev_data(cat_gev, D, names_gev):
    """The fonction adds objects found only in GeV.
    
    D(dictionary) - a dictionary with repeated objects from TeV and GeV
    name_gev(list) - names of GeV columns
    """
    k = 0
    data = []
    
    for j in range(len(cat_gev)):
        if not(j in D.values()):
            data.append([])
            for i in range(len(names_gev)):
                data[k].append(cat_gev[names_gev[i]][j])
            k = k+1
            
    df_only_gev = pd.DataFrame(data = data, columns = names_gev)
        
    return df_only_gev

In [ ]:
data = []
common_data = create_common_data(cat_tev, cat_gev, D, names_common)
print("Added common values = " + str(len(common_data)))
only_tev_data = create_only_tev_data(cat_tev, D, names_tev)
print("Added values which can be found only in grammacat.fits (TeV) = " + str(len(only_tev_data)))
only_gev_data = create_only_gev_data(cat_gev, D, names_gev)
print("Added values which can be found only in gll_psc_v16.fit (GeV) = " + str(len(only_gev_data)))

print("Size of grammacat.fits (TeV) = "  + str(len(cat_tev)))
print("Size of gll_psc_v16.fits (GeV) = "  + str(len(cat_gev)))

In [ ]:
common_data.head()

In [ ]:
only_tev_data.head()

In [ ]:
only_gev_data.head()